# Mestrado em Engenharia Eletrotécnica e Computadores

# Computação Gráfica e Realidade Aumentada-2021/2022

## 3nd Assignment-Exercise 1
## Stereoscopic using anaglyph approach

#### Alexandre Marques - 2018306089
#### Telmo Cunha - 2018308321 

## Exercise C.1 

Neste exercício é nos pedido para utilizar a técnica de visualização estereoscópica com anáglifos, de forma a permitir a visualização do cenário criados nos trabalhos anteriores, no formato 3D. Para isto com base nas fórmulas presentes na ficha 12 e como a utilização da câmera definida para o cenário, criou-se as duas câmeras, posicionado lado a lado, necessárias para criar a câmera estereoscópica, mantendo o ponto de observação e a mesma direção up.
Após a criação de ambas as câmeras, definiu-se as máscaras para cada câmera, sendo que cada um contêm a sua correspondente coloração, como também o seu correspondente drawit, realizando a eliminação do buffer antes da declaração de cada máscara.





In [ ]:
%%html               
<script src="http://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
<script src="http://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script>
<script src="http://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
<script src='https://git.io/glm-js.min.js'></script>


In [ ]:
%%html
<script id="my-vertex-shader" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
uniform mat4 MVP;

varying  vec3 ex_Color;

void main() {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader" type="x-shader/x-fragment">
precision mediump float;

varying  vec3 ex_Color;

void main() {
  
    gl_FragColor = vec4(ex_Color,1.0);
}
</script>

In [ ]:
%%html
<script id="my-vertex-shaderC" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
uniform mat4 MVP;
uniform vec3 un_Color;
varying  vec3 ex_Color;

void main(void) {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = un_Color;
}
</script>

In [ ]:
%%html
<script id="my-vertex-shaderT" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
attribute vec2 in_texcoords;
uniform mat4 MVP;

varying  vec3 ex_Color;
varying  vec2 vTextureCoord;

void main() {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
    vTextureCoord = in_texcoords;
    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shaderT" type="x-shader/x-fragment">
precision mediump float;
varying  vec2 vTextureCoord;
varying  vec3 ex_Color;

uniform sampler2D uSampler;

void main() {
     gl_FragColor = texture2D(uSampler, vTextureCoord);
}
</script>

In [ ]:
%%html
<script id="my-vertex-shaderL" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
attribute  vec3 vertexnormal;
attribute  vec2 in_texcoords;

uniform mat4 MVP;
uniform mat4 modelview;
uniform mat3 normalMatrix;

varying  vec4 Position;
varying  vec3 ex_Color;
varying  vec3 normal;
varying  vec2 vTextureCoord;


void main() {
    
    Position= modelview *vec4(in_Position,1.0);
    normal= normalize(normalMatrix * vertexnormal);
    
    
    gl_Position = MVP * vec4(in_Position, 1.0);

    ex_Color = in_Color;
    vTextureCoord = in_texcoords;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shaderL" type="x-shader/x-fragment">
precision mediump float;

//uniform vec3 ambient;
uniform vec3 lightcolor;
uniform vec3 lightposition;
uniform vec3 eyeposition;
uniform float shininess;
uniform float strenght;
uniform float constantattenuation;
uniform float linearattenuation;
uniform float quadraticattenuation;
uniform sampler2D uSampler;

varying vec3 normal;
varying vec3 ex_Color;
varying vec4 Position;
varying vec2 vTextureCoord;

void main() {
    
    vec3 ambient= vec3(0.5,0.5,0.5); 
    
    vec3 lightdirection=lightposition-vec3(Position);
    float lightdistance=length(lightdirection);
    lightdirection=lightdirection/lightdistance;
    
    vec3 eyedirection =eyeposition-vec3(Position);
    //float eyedistance=length(eyedirection);
    //eyedirection= eyedirection/eyedistance;
    
    vec3 halfvector=normalize(lightdirection+eyedirection);
    
    float attenuation=1.0/(constantattenuation+linearattenuation*lightdistance+quadraticattenuation*lightdistance*lightdistance);
    
    float diffuse = max(0.0, dot(normal, lightdirection));
    float specular= max(0.0, dot(normal,halfvector));
    
    if(diffuse==0.0){
        specular=0.0;
    }
    else{
        specular=pow(specular,shininess)*strenght;
    }
    //vec3 phonglight= ambient + diffuse + specular;
    
    vec3 scatteredLight= ambient + lightcolor * diffuse *attenuation;
    vec3 reflectedlight= lightcolor * specular *attenuation ;//strenght;
    
    //vec4 textcolor=texture2D(uSampler, vTextureCoord);
    
    vec3 rgb=min(( ex_Color.rgb* scatteredLight)+reflectedlight,vec3(1.0));
    //vec3 rgb= min(ex_Color.rgb * phonglight,vec3(1.0));
    gl_FragColor = vec4(rgb ,1.0)*texture2D(uSampler, vTextureCoord);
}
</script>

In [ ]:
%%js
class CGRAtexture{
    constructor(glcontext){
        this.gl = glcontext;
        this.textureid = this.gl.createTexture();
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.level = 0;
        this.internalFormat = this.gl.RGBA;
        this.width = 2;
        this.height = 2;
        this.border = 0;
        this.srcFormat = this.gl.RGBA;
        this.srcType = this.gl.UNSIGNED_BYTE;
        this.pixels = new Uint8Array([255, 255, 255, 255, 
                                      0,0,0,255,  
                                      0,0,0,255, 
                                      255,255,255,255]);  
        this.gl.texImage2D(this.gl.TEXTURE_2D, this.level, this.internalFormat,
                this.width, this.height, this.border, this.srcFormat, this.srcType,
                this.pixels);
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_S, this.gl.CLAMP_TO_EDGE);
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_T, this.gl.CLAMP_TO_EDGE);
        //this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MAG_FILTER, this.gl.LINEAR);
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MAG_FILTER, this.gl.NEAREST);
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MIN_FILTER, this.gl.LINEAR);
    }
    
    loaded(){
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.gl.texImage2D(this.gl.TEXTURE_2D, 0, this.internalFormat,
                this.srcFormat, this.srcType, this.pixels);
        var ispowerof2 = ((this.pixels.width & 1) + (this.pixels.height & 1))==0;
        console.log("width="+this.pixels.width+" height="+this.pixels.height);
        if (ispowerof2) {
           // Yes, it's a power of 2. Generate mipmaps.
           this.gl.generateMipmap(this.gl.TEXTURE_2D);
        } else {
           // No, it's not a power of 2. Turn off mipmaps and set
           // wrapping to clamp to edge
           this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_S, this.gl.CLAMP_TO_EDGE);
           this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_T, this.gl.CLAMP_TO_EDGE);
            this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MAG_FILTER, this.gl.LINEAR);
           this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MIN_FILTER, this.gl.LINEAR);
        }
    }
    load(url){
        this.pixels = new Image();
        this.pixels.src = url;
        this.pixels.onload = () => this.loaded();
    }
}

In [ ]:
%%js
class cube extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        this.vColors = [];
        this.vPositions = [];
        this.texCoords = [];
        
        this.Index=0;
        var r=1.0;
        var g=0.5;
        var b= 0.6;
        this.positions =
            [ glm.vec3(-1.0, -1.0, 1.0),
              glm.vec3(-1.0, 1.0, 1.0),
              glm.vec3(1.0, 1.0, 1.0),
              glm.vec3(1.0, -1.0, 1.0),
              glm.vec3(-1.0, -1.0, -1.0),
              glm.vec3(-1.0, 1.0, -1.0),
              glm.vec3(1.0, 1.0, -1.0),
              glm.vec3(1.0, -1.0, -1.0)]; 
        
        this.colors = [
              glm.vec3(r, g, b),
              glm.vec3(r, g, b),
              glm.vec3(r, g, b),
              glm.vec3(r, g, b),
              glm.vec3(r, g, b),
              glm.vec3(r, g, b),
              glm.vec3(r, g, b),
              glm.vec3(r, g, b)];
        
        this.tcoords = [
            //Front
            glm.vec2(0.25,0.6666),
            glm.vec2(0.25,0.3333),
            glm.vec2(0.50,0.3333),
            glm.vec2(0.25,0.6666),
            glm.vec2(0.50,0.3333),
            glm.vec2(0.50,0.6666),
            
            //Botton
            glm.vec2(0.50,0.6666),
            glm.vec2(0.50,0.3333),
            glm.vec2(0.75,0.3333),
            glm.vec2(0.50,0.6666),
            glm.vec2(0.75,0.3333),
            glm.vec2(0.75,0.6666),
            
            //Back
            glm.vec2(0.75,0.6666),
            glm.vec2(0.75,0.3333),
            glm.vec2(1.00,0.3333),
            glm.vec2(0.75,0.6666),
            glm.vec2(1.00,0.3333),
            glm.vec2(1.00,0.6666),
            
            //left
            glm.vec2(0.50,0.3333),
            glm.vec2(0.50,0.0000),
            glm.vec2(0.75,0.0000),
            glm.vec2(0.50,0.3333),
            glm.vec2(0.75,0.0000),
            glm.vec2(0.75,0.3333),
            
            //rigth
            glm.vec2(0.50,1.0000),
            glm.vec2(0.50,0.6666),
            glm.vec2(0.75,0.6666),
            glm.vec2(0.50,1.0000),
            glm.vec2(0.75,0.6666),
            glm.vec2(0.75,1.0000),
            
            //Top
            glm.vec2(0.00,0.6666),
            glm.vec2(0.00,0.3333),
            glm.vec2(0.25,0.3333),
            glm.vec2(0.00,0.6666),
            glm.vec2(0.25,0.3333),
            glm.vec2(0.25,0.6666)
        ];
        
      
        this.quad( 1, 0, 3, 2, 0);
        this.quad( 2, 3, 7, 6, 6);
        this.quad( 3, 0, 4, 7, 12);
        this.quad( 6, 5, 1, 2, 18);
        this.quad( 4, 5, 6, 7, 24);
        this.quad( 5, 4, 0, 1, 30);
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        var a = new Float32Array(this.vPositions);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        a = new Float32Array(this.vColors);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        a=new Float32Array(this.texCoords);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        //console.log("texc=" + this.texCoords+ "  a= "+ a + " len= "+a.length)
        
        this.numvertices = 36;
    }
    
     quad(a, b, c, d, side)
    {
  
        this.vColors = [].concat(this.vColors,this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[side].array);

        this.vColors = [].concat(this.vColors, this.colors[b].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[b].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[side+1].array);
             
        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[side+2].array);

        this.vColors = [].concat(this.vColors, this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[side+3].array);

        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[side+4].array);

        this.vColors = [].concat(this.vColors,this.colors[d].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[d].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[side+5].array);

    }
    
}

class cubeT extends cube{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat){
        this.shaderprog.startUsing();
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}

In [ ]:
%%html
    
<script id="myapp">
class cone extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        this.numvertices = 60;//60
        var disco = [];
        var actualvertices = [];
        var ind;
        
        var r = [1.0, 0.0, 0.0];
        var g = [0.0, 1.0, 0.0];
        var b = [0.0, 0.0, 1.0];
                 
        var colors = [];
               
        var numlados = this.numvertices /6;
        var offsetx= 1/numlados;
        var textcoord = [];
        
               
        for (var i=-1.0; i<=-1.0; i+=1){
            console.log(i);
            ind = 0;
            var raio = 1.0;
            disco[ind] = [];
            for (var j=-Math.PI/2.0; j<=3*Math.PI/2.0; j+=2.0*Math.PI/10)
                disco[ind] = disco[ind].concat(raio*Math.cos(j),raio*Math.sin(j),i);
        }
        
        //# Unir pontos dos discos ao ponto extremo
        var offset=0;
        for (var j=0; j<30; j+=3){
                actualvertices = actualvertices.concat(disco[0][j],disco[0][j+1],disco[0][j+2]);  
                actualvertices = actualvertices.concat(0,0,1); 
                actualvertices = actualvertices.concat(disco[0][j+3],disco[0][j+4],disco[0][j+5]);
                colors = colors.concat(r,g,b);
            
                //text coords (pontos externos do disco ao apex,(topo))
                textcoord = textcoord.concat([offset, 0.0]);
                textcoord = textcoord.concat([offsetx, 1.0]);
                textcoord = textcoord.concat([offset + offsetx, 0.0]);
                offset=offsetx+offset;
        }
        
        //# Unir pontos dos discos ao centro
        for (var j=0; j<30; j+=3){
                actualvertices = actualvertices.concat(disco[0][j],disco[0][j+1],disco[0][j+2]);  
                actualvertices = actualvertices.concat(0,0,-1); 
                actualvertices = actualvertices.concat(disco[0][j+3],disco[0][j+4],disco[0][j+5]);
                colors = colors.concat(r,g,b);
            
                textcoord= textcoord.concat([0.00,0.00]);//ponto de cima 1
                textcoord= textcoord.concat([0.5,0.5]);//ponto de cima 2
                textcoord= textcoord.concat([1.00,0.00]);// ponto de baixo 2
        }
               
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(textcoord), this.gl.STATIC_DRAW);
            
        
    }
}


class coneT extends cone{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat){
        this.shaderprog.startUsing();
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}


</script>

In [ ]:
%%html
    
<script id="myapp">
class cilindro extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        this.numvertices = 180;
        var disco = [];
        var actualvertices = [];
        var colors= [];
        var ind;
        
        var r = [1.0, 0.0, 0.0];
        var g = [0.0, 1.0, 0.0];
        var b = [0.0, 0.0, 1.0];
        
        var numlados = this.numvertices/12;
        var offsetx= 1.0/numlados;
        var textcoord = [];
        
        for (var i=-1.0; i<=1.0; i+=2){
            if(i == -1.0)
                ind = 0.0;
            else 
                ind = 1.0
            
            var raio = 1.0;
            disco[ind] = [];
            for (var j=-Math.PI/2.0; j<=3*Math.PI/2.0; j+=2.0*Math.PI/10)
                disco[ind] = disco[ind].concat(raio*Math.cos(j),raio*Math.sin(j),i);
            
        }
        // Unir pontos entre discos
        var offset=0;
        for (var i=0; i<1 ; i++){
            //cada ciclo desenha uma face
            for (var j=0; j<30; j+=3){
                

                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto do disco acima
                actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto seguinte
                colors = colors.concat(r,g,b);
                
                textcoord= textcoord.concat([offset,1.00]);//ponto de cima
                textcoord= textcoord.concat([offset,0.00]);//ponto
                textcoord= textcoord.concat([(offset+offsetx),0.00]);//ponto seguinte
                
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i+1][j+3],disco[i+1][j+4],disco[i+1][j+5]); // 1 ponto seguinte
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto do disco acima
                colors = colors.concat(r,g,b);
                
                textcoord= textcoord.concat([offset,1.00]);//ponto de cima 1
                textcoord= textcoord.concat([(offset+offsetx),1.00]);//ponto de cima 2
                textcoord= textcoord.concat([(offset+offsetx),0.00]);// ponto de baixo 2
                
                //console.log("Debug:coord textura:");
                //console.log(textcoord);
                
                offset=offset+offsetx;
            }
            
        }

        // Unir pontos dos discos ao centro
        for (var i=0; i<=1 ; i++){
            for (var j=0; j<30; j+=3){
                if(i==0){
                    actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]);  
                    actualvertices = actualvertices.concat(0,0,-1); 
                    actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]);
                    colors = colors.concat(r,g,b);
                    
                    textcoord= textcoord.concat([0.00,0.00]);//ponto de cima 1
                    textcoord= textcoord.concat([0.5,0.5]);//ponto de cima 2
                    textcoord= textcoord.concat([1.00,0.00]);// ponto de baixo 2
                }
                if(i==1){
                    actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]);  
                    actualvertices = actualvertices.concat(0,0,1); 
                    actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]);
                    colors = colors.concat(r,g,b);
                    
                    textcoord= textcoord.concat([0.00,0.00]);//ponto de cima 1
                    textcoord= textcoord.concat([0.5,0.5]);//ponto de cima 2
                    textcoord= textcoord.concat([1.00,0.00]);// ponto de baixo 2
                }
                
            }
        }
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.texturebuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texturebuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(textcoord), this.gl.STATIC_DRAW);
    }
}
class cilindroT extends cilindro{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat){
        this.shaderprog.startUsing();
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texturebuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.textureLocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}




</script>

In [ ]:
%%html
    
<script id="myapp">

class esfera extends CGRAobject{
    constructor(glcontext, r, g, b){
        super(glcontext); // initialize the parent class
        
        this.numvertices = 2160;
        var centro = [0.0, 0.0, 0.0];
        var disco = [];
        var colors = [];
        var actualvertices = [];
        const raioConstante = 1.0;
        //var ind;
                
        var numlados = this.numvertices /6;
        var offsetx= 1.0/numlados;
        var offsety= 1.0/numlados;
        var textcoord = [];
        
        
        for (var ind=0; ind<=10; ind++){
            disco[ind] = [];
            
            var raio = raioConstante * Math.cos((Math.PI/10*ind) - Math.PI/2);
            
            for (var j=-Math.PI/2.0; j<=3.0*Math.PI; j+=1.0*Math.PI/10.0)
                disco[ind] = disco[ind].concat(raio*Math.cos(j),raio*Math.sin(j),Math.sin((Math.PI/10*ind) - Math.PI/2));
    
        }
        
        for (var i=0; i<disco.length-1 ; i++){
            var offset1=0;
            var offset2=0;
            for (var j=0; j<disco[0].length; j+=3){
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto do disco acima
                actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto seguinte mesmo disco
                colors = colors.concat(r,g,b);
                
                textcoord=textcoord.concat(offset1,offset2+offsety);
                textcoord=textcoord.concat(offset1,offset2);
                textcoord=textcoord.concat(offset1+offsetx,offsety);
                
                
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i+1][j+3],disco[i+1][j+4],disco[i+1][j+5]); // 1 ponto seguinte mesmo disco
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto do disco abaixo
                colors = colors.concat(r,g,b);
                
                textcoord=textcoord.concat(offset1,offsety);
                textcoord=textcoord.concat(offset1+offsetx,offsety);
                textcoord=textcoord.concat(offset1,-offsety);
                
                offset1=offset1+offsetx;
                offset2=offset2+offsety;
                
            }
        }
        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.texturebuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texturebuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(textcoord), this.gl.STATIC_DRAW);
    }
}



</script>

In [ ]:
%%html
<script id="myapp">

class cubeTeste extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        this.vColors = [];
        this.vPositions = [];
        this.texCoords = [];
        this.normals=[];
        
        this.Index=0;
        //var r=1.0;
        //var g=0.5;
        //var b= 0.6;
        this.positions =
            [ glm.vec3(-1.0, -1.0, 1.0),
              glm.vec3(-1.0, 1.0, 1.0),
              glm.vec3(1.0, 1.0, 1.0),
              glm.vec3(1.0, -1.0, 1.0),
              glm.vec3(-1.0, -1.0, -1.0),
              glm.vec3(-1.0, 1.0, -1.0),
              glm.vec3(1.0, 1.0, -1.0),
              glm.vec3(1.0, -1.0, -1.0)]; 
        
        this.colors = [
              glm.vec3(1.0, 0.0, 0.0),
              glm.vec3(1.0, 0.0, 0.0),
              glm.vec3(1.0, 0.0, 0.0),
              glm.vec3(1.0, 0.0, 0.0),
              glm.vec3(1.0, 0.0, 0.0),
              glm.vec3(1.0, 0.0, 0.0),
              glm.vec3(1.0, 0.0, 0.0),
              glm.vec3(1.0, 0.0, 0.0)];
        
        this.tcoords = [
            //Front
            glm.vec2(0.25,0.6666),
            glm.vec2(0.25,0.3333),
            glm.vec2(0.50,0.3333),
            glm.vec2(0.25,0.6666),
            glm.vec2(0.50,0.3333),
            glm.vec2(0.50,0.6666),
            
            //Botton
            glm.vec2(0.50,0.6666),
            glm.vec2(0.50,0.3333),
            glm.vec2(0.75,0.3333),
            glm.vec2(0.50,0.6666),
            glm.vec2(0.75,0.3333),
            glm.vec2(0.75,0.6666),
            
            //Back
            glm.vec2(0.75,0.6666),
            glm.vec2(0.75,0.3333),
            glm.vec2(1.00,0.3333),
            glm.vec2(0.75,0.6666),
            glm.vec2(1.00,0.3333),
            glm.vec2(1.00,0.6666),
            
            //left
            glm.vec2(0.50,0.3333),
            glm.vec2(0.50,0.0000),
            glm.vec2(0.75,0.0000),
            glm.vec2(0.50,0.3333),
            glm.vec2(0.75,0.0000),
            glm.vec2(0.75,0.3333),
            
            //rigth
            glm.vec2(0.50,1.0000),
            glm.vec2(0.50,0.6666),
            glm.vec2(0.75,0.6666),
            glm.vec2(0.50,1.0000),
            glm.vec2(0.75,0.6666),
            glm.vec2(0.75,1.0000),
            
            //Top
            glm.vec2(0.00,0.6666),
            glm.vec2(0.00,0.3333),
            glm.vec2(0.25,0.3333),
            glm.vec2(0.00,0.6666),
            glm.vec2(0.25,0.3333),
            glm.vec2(0.25,0.6666)
        ];
        
        this.quad( 1, 0, 3, 2, 0);
        this.quad( 2, 3, 7, 6, 6);
        this.quad( 3, 0, 4, 7, 12);
        this.quad( 6, 5, 1, 2, 18);
        this.quad( 4, 5, 6, 7, 24);
        this.quad( 5, 4, 0, 1, 30);
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        var a = new Float32Array(this.vPositions);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        a = new Float32Array(this.vColors);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        a=new Float32Array(this.texCoords);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.normalbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalbuffer);  
        a=new Float32Array(this.normals);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        //console.log("texc=" + this.texCoords+ "  a= "+ a + " len= "+a.length)
        
        this.numvertices = 36;
    }
    
     quad(a, b, c, d, side)
    {
        var ux = this.positions[a].x - this.positions[b].x;
        var uy = this.positions[a].y - this.positions[b].y;
        var uz = this.positions[a].z - this.positions[b].z;
        var vx = this.positions[a].x - this.positions[c].x;
        var vy = this.positions[a].y - this.positions[c].y;
        var vz = this.positions[a].z - this.positions[c].z;
        var wx = this.positions[a].x - this.positions[d].x;
        var wy = this.positions[a].y - this.positions[d].y;
        var wz = this.positions[a].z - this.positions[d].z;
        var n1= glm.normalize(glm.cross(glm.vec3(uz,ux,ux), glm.vec3(vz,vx,vx)));
        var n2= glm.normalize(glm.cross(glm.vec3(vz,vx,vx), glm.vec3(wz,wx,wx)));
        
  
        this.vColors = [].concat(this.vColors,this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[side].array);
        this.normals = [].concat(this.normals,n1.array);
        //this.Normals = [].concat(this.Normals,this.normals[side].array);

        this.vColors = [].concat(this.vColors, this.colors[b].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[b].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[side+1].array);
        this.normals = [].concat(this.normals,n1.array);
        //this.Normals = [].concat(this.Normals,this.normals[side+1].array);
             
        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[side+2].array);
        this.normals = [].concat(this.normals,n1.array);
        //this.Normals = [].concat(this.Normals,this.normals[side+2].array);

        this.vColors = [].concat(this.vColors, this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[side+3].array);
        this.normals = [].concat(this.normals,n2.array);
        //this.Normals = [].concat(this.Normals,this.normals[side+3].array);

        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[side+4].array);
        this.normals = [].concat(this.normals,n2.array);
        //this.Normals = [].concat(this.Normals,this.normals[side+4].array);

        this.vColors = [].concat(this.vColors,this.colors[d].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[d].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[side+5].array);
        this.normals = [].concat(this.normals,n2.array);
        //this.Normals = [].concat(this.Normals,this.normals[side+5].array);

    }
    
}

class cubePLight extends cubeTeste{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat){
        this.shaderprog.startUsing();
        
        //console.log("Normals:\n");
        //console.log(this.normals);
        
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalbuffer);
        this.NormalLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "vertexnormal");
        this.gl.vertexAttribPointer(this.NormalLocation, // Attribute location
                           3, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0,//2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.NormalLocation);
        
        var MV= this.gl.getUniformLocation(this.shaderprog.shaderProgram,"modelview");
        var modelview =this.modelMat;
        this.gl.uniformMatrix4fv(MV,false,modelview.array);
        
        var nmatrixnew =glm.mat3(this.modelMat);
        var nmatrix= this.gl.getUniformLocation(this.shaderprog.shaderProgram,"normalMatrix");
        this.gl.uniformMatrix3fv(nmatrix,false,nmatrixnew.array);
        
        /*var ambient= this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Ambient");
        var ambiente= glm.vec3(0.6,0.6,0.6);
        this.gl.uniform3fv(ambient,ambiente.array);*/
        
        var lightcolor= this.gl.getUniformLocation(this.shaderprog.shaderProgram,"lightcolor");
        var light = glm.vec3(1.0,1.0,1.0);
        this.gl.uniform3fv(lightcolor,light.array);
        
        var lightposition= this.gl.getUniformLocation(this.shaderprog.shaderProgram,"lightposition");
        var lightpos=glm.vec3(0.0,.0,0.0);
        this.gl.uniform3fv(lightposition,lightpos.array);
        
        var eyeposition= this.gl.getUniformLocation(this.shaderprog.shaderProgram,"eyeposition");
        var eyepos=glm.vec3(2,-1.0,3);
        this.gl.uniform3fv(eyeposition,eyepos.array);
        
        var shininess=this.gl.getUniformLocation(this.shaderprog.shaderProgram,"shininess");
        var shi=4;
        this.gl.uniform1f(shininess,shi);
        
        var strength = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"strenght");
        var strong=10;
        this.gl.uniform1f(strength,strong);
        
        var constantattenuation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"constantattenuation");
        var constatt=0.0;
        this.gl.uniform1f(constantattenuation,constatt);
        
        var linearattenuation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"linearattenuation");
        var linearatt=0.0;
        this.gl.uniform1f(linearattenuation,linearatt);
        
        var quadraticattenuation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"quadraticattenuation");
        var quadraticatt=0.21;
        this.gl.uniform1f(quadraticattenuation,quadraticatt);
        
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        

        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
        
     }  
}



</script>

In [ ]:
%%html
    
<script id="myapp">

class CompoundObject extends CGRAobject{
    array=[];
    obj;
    
    //metodo para inserir objeto
    insertObject(objeto, transformation = glm.mat4(1.0)){
        this.obj = objeto;
        this.obj.setModelTransformation(transformation);
        this.array = this.array.concat(this.obj);
    }
  
    setShader(shader){
        for (var i=0; i<this.array.length; i++)
            this.array[i].shaderprog = shader;
    }
   
    drawit(V,P,parent=glm.mat4(1.0)){
        var trans;
        for(var i=0;i<this.array.length; i++){
            trans = parent['*'](this.array[i].modelMat);
            var aux = this.array[i].modelMat;
            this.array[i].modelMat = trans;
            this.array[i].drawit(V,P);
            this.array[i].modelMat = aux;
        }
    }
  
};

</script>

In [ ]:
%%html    
<script id="myapp">

class esferatext extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        this.numvertices = 2160;
        var centro = [0.0, 0.0, 0.0];
        var disco = [];
        var colors = [];
        var actualvertices = [];
        const raioConstante = 1.0;
        //var ind;
        
        var r = [1.0, 0.0, 0.0];
        var g = [0.0, 1.0, 0.0];
        var b = [0.0, 0.0, 1.0];
        
        var textcoord = [];
               
        for (var ind=0; ind<=10; ind++){
            disco[ind] = [];
            
            var raio = raioConstante * Math.cos((Math.PI/10*ind) - Math.PI/2);
            
            for (var j=-Math.PI/2.0; j<=3.0*Math.PI; j+=1.0*Math.PI/10.0)
                disco[ind] = disco[ind].concat(raio*Math.cos(j),raio*Math.sin(j),Math.sin((Math.PI/10*ind) - Math.PI/2));
    
        }
        
        var offsety = 1.0/(disco.length);
        var offset2 = 0.0;
        for (var i=0; i<disco.length-1 ; i++){
            
            var offset1 = 0.0;
            var offsetx = 1.0/20; 
            
            for (var j=0; j<disco[0].length; j+=3){
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto do disco acima
                actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]);  // 1 ponto
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]);  // 1 ponto seguinte mesmo disco
                colors = colors.concat(r,g,b);
                
                textcoord=textcoord.concat([offset1,(offset2+offsety)]); //ponto disco2
                textcoord=textcoord.concat([offset1,offset2]); //ponto disco 1
                textcoord=textcoord.concat([(offset1+offsetx),offset2]); //ponto2 disco 1
   
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i+1][j+3],disco[i+1][j+4],disco[i+1][j+5]); // 1 ponto seguinte mesmo disco
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]);  // 1 ponto do disco abaixo
                colors = colors.concat(r,g,b);
                
                textcoord=textcoord.concat([offset1,(offset2+offsety)]); //ponto disco2
                textcoord=textcoord.concat([(offset1+offsetx),(offset2+offsety)]); //ponto2 disco2
                textcoord=textcoord.concat([(offset1+offsetx),offset2]); //ponto 2 disco1
                
                offset1=offset1+offsetx;
           }
            offset2=offset2+offsety;
        }     
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.texturebuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texturebuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(textcoord), this.gl.STATIC_DRAW);
    }
}

class esferaT extends esferatext{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat){
        this.shaderprog.startUsing();
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texturebuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.textureLocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}


</script>

## Cenario 

In [ ]:
%%html
    
<canvas id="my5Canvas" width="1280" height="720" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class my5app extends DEECapp{
    counter = 0;
    
    initialize(){
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
        
        var vertsrcC = document.getElementById("my-vertex-shaderC").text;
        
        this.shaderprogC = new DEECshader(this.gl);
        this.shaderprogC.srcShaders(vertsrcC,fragsrc);
        
        //# vertex e fragment shader para a luz 
        var fragsrcL = document.getElementById("my-fragment-shaderL").text;
        var vertsrcL = document.getElementById("my-vertex-shaderL").text;
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        //-------------------------------
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        
        //************************************************************************************
        //#--------- -- Textura Dinamica(Bonus) -------------------------------------
        
        // Create a texture to render to
        this.targetTextureWidth = 512;
        this.targetTextureHeight = 512;
        this.targetTexture = this.gl.createTexture();
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.targetTexture);
        
        this.gl.texImage2D(this.gl.TEXTURE_2D, 0, this.gl.RGBA,
                  this.targetTextureWidth, this.targetTextureHeight, 0,
                  this.gl.RGBA, this.gl.UNSIGNED_BYTE, null);
        
         // set the filtering so we don't need mips
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MIN_FILTER, this.gl.LINEAR);
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_S, this.gl.CLAMP_TO_EDGE);
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_T, this.gl.CLAMP_TO_EDGE);

        // Create and bind the framebuffer
        this.fb = this.gl.createFramebuffer();
        this.gl.bindFramebuffer(this.gl.FRAMEBUFFER, this.fb);

        this.gl.framebufferTexture2D(this.gl.FRAMEBUFFER, this.gl.COLOR_ATTACHMENT0, 
                                   this.gl.TEXTURE_2D, this.targetTexture, 0);

        // create a depth renderbuffer
        this.depthBuffer = this.gl.createRenderbuffer();
        this.gl.bindRenderbuffer(this.gl.RENDERBUFFER, this.depthBuffer);

        // make a depth buffer and the same size as the targetTexture
        this.gl.renderbufferStorage(this.gl.RENDERBUFFER, this.gl.DEPTH_COMPONENT16, 
                                  this.targetTextureWidth, this.targetTextureHeight);
        this.gl.framebufferRenderbuffer(this.gl.FRAMEBUFFER, this.gl.DEPTH_ATTACHMENT, 
                                      this.gl.RENDERBUFFER, this.depthBuffer);
        
        
        //*****************************************************************
        //# ------------------- Chão --------------------------
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("newGround.png");
        
        this.chao = new cubeT(this.gl);
        this.chao.settexture(this.texture);
        this.chao.setShader(this.shaderprogT);
        
        //# ------------------- Céu --------------------------
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("skyTexture.jpg");
        this.daySky = new cubeT(this.gl);
        this.daySky.settexture(this.texture);
        this.daySky.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("newNigth.png");
        this.nightSky = new cubeT(this.gl);
        this.nightSky.settexture(this.texture);
        this.nightSky.setShader(this.shaderprogT);
                 
    
        //***************************************************************
        //# ------------------ Arvore ---------------------------------
        
        this.pineTree = new CompoundObject;
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("treeleaf.png");
        this.pineTreeCrown = new coneT(this.gl);
        this.pineTreeCrown.settexture(this.texture);
        this.pineTreeCrown.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("trunk.png");
        this.pineTreeTrunk = new cilindroT(this.gl);
        this.pineTreeTrunk.settexture(this.texture);
        this.pineTreeTrunk.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("Xmaspaper.jpg");
        
        this.xmasBall1 = new esferaT(this.gl);
        this.xmasBall1.settexture(this.texture);
        this.xmasBall1.setShader(this.shaderprogT);
        
        this.xmasBall2 = new esferaT(this.gl);
        this.xmasBall2.settexture(this.texture);
        this.xmasBall2.setShader(this.shaderprogT);
        
        this.xmasBall3 = new esferaT(this.gl);
        this.xmasBall3.settexture(this.texture);
        this.xmasBall3.setShader(this.shaderprogT);
        
        this.xmasBall4 = new esferaT(this.gl);
        this.xmasBall4.settexture(this.texture);
        this.xmasBall4.setShader(this.shaderprogT);
           
        var grau90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        
        var crownPos = glm.vec3(-5.0,12.0,-5.0);
        
        //# Copa da arvore
        var scale = glm.scale(glm.vec3(4,4,6));
        var transformation = glm.translate(glm.vec3(crownPos[0],crownPos[1],crownPos[2]));
        this.pineTree.insertObject(this.pineTreeCrown, transformation['*'](grau90)['*'](scale));
        
        //# Tronco
        var scale = glm.scale(glm.vec3(1,1,5));
        var transformation = glm.translate(glm.vec3(crownPos[0],crownPos[1]-10,crownPos[2]));
        this.pineTree.insertObject(this.pineTreeTrunk, transformation['*'](grau90)['*'](scale));       
        
        //# bolas de natal
        var scale = glm.mat4(glm.mat3(1.0));
        var transformation = glm.translate(glm.vec3(crownPos[0]+2,crownPos[1]-3,crownPos[2]+2));
        this.pineTree.insertObject(this.xmasBall1, transformation['*'](scale));       
              
        var transformation = glm.translate(glm.vec3(crownPos[0]+1,crownPos[1]+2,crownPos[2]+1));
        this.pineTree.insertObject(this.xmasBall2, transformation['*'](scale)); 
        
        var transformation = glm.translate(glm.vec3(crownPos[0]-2,crownPos[1]-3,crownPos[2]-2));
        this.pineTree.insertObject(this.xmasBall3, transformation['*'](scale)); 
        
        var transformation = glm.translate(glm.vec3(crownPos[0]-1,crownPos[1]+2,crownPos[2]-1));
        this.pineTree.insertObject(this.xmasBall4, transformation['*'](scale)); 
              
        this.pineTree.setShader(this.shaderprogT);
        
        //***********************************************************************************************
         //# ------------------ Billboard --------------------------------
        this.billboard = new CompoundObject;
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("metalTexture.jpg");
        this.supportboard1 = new cilindroT(this.gl);
        this.supportboard1.settexture(this.texture);
        this.supportboard1.setShader(this.shaderprogT);
        
        this.supportboard2 = new cilindroT(this.gl);
        this.supportboard2.settexture(this.texture);
        this.supportboard2.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("rngtexture.png");
        this.board = new cubeT(this.gl);
        this.board.settexture(this.texture);
        this.board.setShader(this.shaderprogT);
        
        var grau90 = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        
        //# Board
        var scale = glm.scale(glm.vec3(0.2,3,10));
        var transformation = glm.translate(glm.vec3(5.0,10.0,5.0));
        this.billboard.insertObject(this.board, transformation['*'](scale));
        
        //# Support 1
        var scale = glm.scale(glm.vec3(0.2,0.2,5));
        var transformation = glm.translate(glm.vec3(5.0,5.0,-4));
        this.billboard.insertObject(this.supportboard1, transformation['*'](grau90)['*'](scale));
        
        //# Support 1
        var scale = glm.scale(glm.vec3(0.2,0.2,5));
        var transformation = glm.translate(glm.vec3(5.0,5.0,14));
        this.billboard.insertObject(this.supportboard2, transformation['*'](grau90)['*'](scale));
        
        this.billboard.setShader(this.shaderprogT);
        
        //********************************************
        //# ------- placard com textura dinamica -----
        
        this.placard = new CompoundObject;
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("metaltexture.jpg");
        this.supportplacard1 = new cilindroT(this.gl);
        this.supportplacard1.settexture(this.texture);
        this.supportplacard1.setShader(this.shaderprogT);
        
        this.boardplacard = new cubeT(this.gl);
        //this.boardplacard.settexture(this.texture);
        this.boardplacard.textureid = this.targetTexture;
        this.boardplacard.setShader(this.shaderprogT);
        
        //# Board
        var scale = glm.scale(glm.vec3(5,3,0.2));
        var transformation = glm.translate(glm.vec3(5.0,10.0,5.0));
        this.placard.insertObject(this.boardplacard, transformation['*'](scale));
        
        //# Support 1
        var scale = glm.scale(glm.vec3(0.5,0.5,2));
        var transformation = glm.translate(glm.vec3(5.0,5.0,5.0));
        this.placard.insertObject(this.supportplacard1, transformation['*'](grau90)['*'](scale));
        
        this.placard.setShader(this.shaderprogT);
        
        //***********************************************************
        //# ------------------ bancadas ------------------------------------
        this.stand = new CompoundObject;
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("concreteTexture.jpg");
        this.stand1 = new cubeT(this.gl);
        this.stand1.settexture(this.texture);
        this.stand1.setShader(this.shadeprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("rubber.jpg");
        this.stand2 = new cubeT(this.gl);
        this.stand2.settexture(this.texture);
        this.stand2.setShader(this.shadeprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("concreteTexture.jpg");
        this.stand3 = new cubeT(this.gl);
        this.stand3.settexture(this.texture);
        this.stand3.setShader(this.shadeprogT);
                
        var grau90 = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(1.0,0.0,0.0))); 
        var standLength = 30;
        var standPos = glm.vec3(0.0,0.0,0.0);
                
        //# Stand 1
        var scale = glm.scale(glm.vec3(standLength,1,5));
        var transformation = glm.translate(glm.vec3(standPos[0],standPos[1],standPos[2]));
        this.stand.insertObject(this.stand1, transformation['*'](scale));
        
        //# Stand 2
        var scale = glm.scale(glm.vec3(standLength,1,3));
        var transformation = glm.translate(glm.vec3(standPos[0],standPos[1]+2,standPos[2]-2));
        this.stand.insertObject(this.stand2, transformation['*'](scale));
        
        //# Stand 3
        var scale = glm.scale(glm.vec3(standLength,1,1));
        var transformation = glm.translate(glm.vec3(standPos[0],standPos[1]+4,standPos[2]-4));
        this.stand.insertObject(this.stand3, transformation['*'](scale));
        
               
        this.stand.setShader(this.shaderprogT);
       
        
        //****************************************************************
        //#------------------ Banners (mini placard) --- --------------------------------
        //#Banners 1
        this.finish = new CompoundObject;
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("metalTexture.jpg");
        this.support3 = new cilindroT(this.gl);
        this.support3.settexture(this.texture);
        this.support3.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("newWRC.png");
        this.banner = new cubeT(this.gl);
        this.banner.settexture(this.texture);
        this.banner.setShader(this.shaderprogT);
        
        var grau90 = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));   
        var boardPos = glm.vec3(0,10,0);
        var bannerScale = 3;
        
        //#tela
        var scale = glm.scale(glm.vec3(0.2,3,bannerScale));
        var transformation = glm.translate(glm.vec3(0.0,10.0,0.0));
        this.finish.insertObject(this.banner, transformation['*'](scale));
        
        //# Suporte 1
        var scale = glm.scale(glm.vec3(0.2,0.2,5));
        var transformation = glm.translate(glm.vec3(boardPos[0],boardPos[1]-5,boardPos[2]-bannerScale+1));
        this.finish.insertObject(this.support3, transformation['*'](grau90)['*'](scale));
        
        this.finish.setShader(this.shaderprogT);
        
        
        //#Banners2
        this.finish2 = new CompoundObject;
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("metalTexture.jpg");
        this.support4 = new cilindroT(this.gl);
        this.support4.settexture(this.texture);
        this.support4.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("newFlag.png");
        this.banner2 = new cubeT(this.gl);
        this.banner2.settexture(this.texture);
        this.banner2.setShader(this.shaderprogT);
               
        //# tela
        var scale = glm.scale(glm.vec3(0.2,3,bannerScale));
        var transformation = glm.translate(glm.vec3(0.0,10.0,0.0));
        this.finish2.insertObject(this.banner2, transformation['*'](scale));
        
        //# Suporte 1
        var scale = glm.scale(glm.vec3(0.2,0.2,5));
        var transformation = glm.translate(glm.vec3(boardPos[0],boardPos[1]-5,boardPos[2]-bannerScale+1));
        this.finish2.insertObject(this.support4, transformation['*'](grau90)['*'](scale));
        
        this.finish2.setShader(this.shaderprogT);
        
        //#Banners 3
        this.finish3 = new CompoundObject;
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("metalTexture.jpg");
        this.support5 = new cilindroT(this.gl);
        this.support5.settexture(this.texture);
        this.support5.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("newBf2042.png");
        this.banner3 = new cubeT(this.gl);
        this.banner3.settexture(this.texture);
        this.banner3.setShader(this.shaderprogT);
                
        //# tela
        var scale = glm.scale(glm.vec3(0.2,3,bannerScale));
        var transformation = glm.translate(glm.vec3(0.0,10.0,0.0));
        this.finish3.insertObject(this.banner3, transformation['*'](scale));
        
        //# Suporte 1
        var scale = glm.scale(glm.vec3(0.2,0.2,5));
        var transformation = glm.translate(glm.vec3(boardPos[0],boardPos[1]-5,boardPos[2]-bannerScale+1));
        this.finish3.insertObject(this.support5, transformation['*'](grau90)['*'](scale));
        
        this.finish3.setShader(this.shaderprogT);
               
        //********************************************************************
        //# ------------------ Carros --------------------------------------
        
        this.car2 = new CompoundObject;
        this.car = new CompoundObject;
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("tire.jpg");
        this.roda1 = new cilindroT(this.gl);
        this.roda1.settexture(this.texture);
        this.roda1.setShader(this.shaderprogT);
        
        this.roda2 = new cilindroT(this.gl);
        this.roda2.settexture(this.texture);
        this.roda2.setShader(this.shaderprogT);
        
        this.roda3 = new cilindroT(this.gl);
        this.roda3.settexture(this.texture);
        this.roda3.setShader(this.shaderprogT);
        
        this.roda4 = new cilindroT(this.gl);
        this.roda4.settexture(this.texture);
        this.roda4.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("carRacing.png");
        this.base = new cubeT(this.gl);
        this.base.settexture(this.texture);
        this.base.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("clio.png");
        this.habit = new cubeT(this.gl);
        this.habit.settexture(this.texture);
        this.habit.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("rubber.jpg");
        this.spoiler = new cubeT(this.gl);
        this.spoiler.settexture(this.texture);
        this.spoiler.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("metalTexture.jpg");
        this.supportsp1 = new cilindroT(this.gl);
        this.supportsp1.settexture(this.texture);
        this.supportsp1.setShader(this.shaderprogT);
        this.supportsp2 = new cilindroT(this.gl);
        this.supportsp2.settexture(this.texture);
        this.supportsp2.setShader(this.shaderprogT);
               
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("wiper.jpg");
        this.habitfront = new coneT(this.gl);
        this.habitfront.settexture(this.texture);
        this.habitfront.setShader(this.shaderprogT);
        this.habitback = new coneT(this.gl);
        this.habitback.settexture(this.texture);
        this.habitback.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("carRacing.png");
        this.base2 = new cubeT(this.gl);
        this.base2.settexture(this.texture);
        this.base2.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("clio.png");
        this.habit2 = new cubeT(this.gl);
        this.habit2.settexture(this.texture);
        this.habit2.setShader(this.shaderprogT);
        
        var grau90 = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(0.0,1.0,0.0)));
        var basePos = glm.vec3(-0.1,1,-0.2);
        
        //# Base 
        scale = glm.scale(glm.vec3(2.8,1.0,4));
        transformation = glm.translate(glm.vec3(basePos[0],basePos[1],basePos[2]));
        this.car.insertObject(this.base, transformation['*'](scale));
        this.car2.insertObject(this.base2, transformation['*'](scale));
        
        //# Cabine
        scale = glm.scale(glm.vec3(2.695,1,2));
        transformation = glm.translate(glm.vec3(basePos[0],basePos[1]+1,basePos[2]));
        this.car.insertObject(this.habit, transformation['*'](scale));
        this.car2.insertObject(this.habit2, transformation['*'](scale));
        
        //# Roda 1 
        var scale = glm.scale(glm.vec3(1,1,0.5));
        var transformation = glm.translate(glm.vec3(basePos[0]-1.9,basePos[1]-1,basePos[2]-1.8));
        this.car.insertObject(this.roda1, transformation['*'](grau90)['*'](scale));
        this.car2.insertObject(this.roda1, transformation['*'](grau90)['*'](scale));
                
        //# Roda 2 
        var scale = glm.scale(glm.vec3(1,1,0.5));
        transformation = glm.translate(glm.vec3(basePos[0]+2.1,basePos[1]-1,basePos[2]-1.8));
        this.car.insertObject(this.roda2,transformation['*'](grau90)['*'](scale));
        this.car2.insertObject(this.roda2,transformation['*'](grau90)['*'](scale));
                
        //# Roda 3 
        var scale = glm.scale(glm.vec3(1,1,0.5));
        transformation = glm.translate(glm.vec3(basePos[0]+2.1,basePos[1]-1,basePos[2]+2.2));
        this.car.insertObject(this.roda3,transformation['*'](grau90)['*'](scale));
        this.car2.insertObject(this.roda3,transformation['*'](grau90)['*'](scale));
               
        //# Roda 4 
        var scale = glm.scale(glm.vec3(1,1,0.5));
        transformation = glm.translate(glm.vec3(basePos[0]-1.9,basePos[1]-1,basePos[2]+2.2));
        this.car.insertObject(this.roda4,transformation['*'](grau90)['*'](scale));
        this.car2.insertObject(this.roda4,transformation['*'](grau90)['*'](scale));
        
        
        //#spoiler e seu suporte
        var grausupport = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        var scalesup= glm.scale(glm.vec3(0.2,0.2,1.0));
        transformation= glm.translate(glm.vec3(-2.0,2,3.2))
        this.car.insertObject(this.supportsp1,transformation['*'](grausupport)['*'](scalesup));
        this.car2.insertObject(this.supportsp1,transformation['*'](grausupport)['*'](scalesup));
        
        transformation= glm.translate(glm.vec3(2,2,3.2))
        this.car.insertObject(this.supportsp2,transformation['*'](grausupport)['*'](scalesup));
        this.car2.insertObject(this.supportsp2,transformation['*'](grausupport)['*'](scalesup));
        
        var scalesp= glm.scale(glm.vec3(0.5,0.25,4));
        var transformation= glm.translate(glm.vec3(-0.1,3,3.2));
        this.car.insertObject(this.spoiler,transformation['*'](grau90)['*'](scalesp));
        this.car2.insertObject(this.spoiler,transformation['*'](grau90)['*'](scalesp));
        
        //# improvement of the shape of the car
        scale = glm.scale(glm.vec3(2.695,1,2));
        transformation = glm.translate(glm.vec3(-0.1,2,-1.9));
        this.car.insertObject(this.habitfront, transformation['*'](scale));
        this.car2.insertObject(this.habitfront, transformation['*'](scale));
        
        scale = glm.scale(glm.vec3(2.695,1,2));
        transformation = glm.translate(glm.vec3(-0.1,2,1.9));
        this.car.insertObject(this.habitback, transformation['*'](scale));
        this.car2.insertObject(this.habitback, transformation['*'](scale));
                
        this.car.setShader(this.shaderprogT);
        this.car2.setShader(this.shaderprogT);
        
        //***************************************************************
        // #-------------------------- Lampadas esfericas --------------------------
        this.Groundlamp = new CompoundObject;
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("metaltexture.jpg");
        this.supportlamp = new cilindroT(this.gl);
        this.supportlamp.settexture(this.texture);
        this.supportlamp.setShader(this.shaderprogT);
        
        this.lamp = new esfera(this.gl,[1.0,0.0,0.0],[1.0,0.0,0.0],[1.0,0.0,0.0]);
        this.lamp.setShader(this.shaderprog);
        
        var grau90 = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        
        //#lamp
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(0.0, 7.5, 0.0));
        this.Groundlamp.insertObject(this.lamp, transformation['*'](scale));
        
        //# Support 
        var scale = glm.scale(glm.vec3(0.1,0.1,2));
        var transformation = glm.translate(glm.vec3(0.0, 5.0, 0.0));
        this.Groundlamp.insertObject(this.supportlamp, transformation['*'](grau90)['*'](scale));
        
        // #-------------------------- Lampadas cubicas --------------------------
        this.Streetlamp = new CompoundObject;
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("metaltexture.jpg");
        this.supportlamp1 = new cilindroT(this.gl);
        this.supportlamp1.settexture(this.texture);
        this.supportlamp1.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("lamp.png");
        this.lamp1 = new cubePLight(this.gl);
        this.lamp1.settexture(this.texture);
        this.lamp1.setShader(this.shaderprogL);
        
        var grau90 = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        
        //#lamp
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(0.0, 7.5, 0.0));
        this.Streetlamp.insertObject(this.lamp1, transformation['*'](scale));
        
        //# Support 
        var scale = glm.scale(glm.vec3(0.1,0.1,2));
        var transformation = glm.translate(glm.vec3(0.0, 5.0, 0.0));
        this.Streetlamp.insertObject(this.supportlamp1, transformation['*'](grau90)['*'](scale));
              
        
        //*********************************************************************
        
        //resolução do canvas, 1280*720, 16/9
        this.projectionM = glm.perspective(glm.radians(45),16/9,0.1,1000);
                
        this.posx = -50;
        this.posz = -60;
        this.theta = 6*Math.PI/20;
        
        this.viewM = glm.lookAt(glm.vec3(this.posx,0,this.posz),
                                glm.vec3(Math.sin(this.theta)+this.posx,0,
                                         Math.cos(this.theta)+this.posz),
                                glm.vec3(0,1,0));
                     
                       
       //************************************************************* 
        this.carPos = glm.vec3(0.0,0.0,0.0);
        
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
        this.viewpoint = true;
      
    }
    
    
    //=========================================================================================================================
    //=========================================================================================================================
    
    changeCenario(p){
        switch(p){
            case 1: //dia
            
                var rotate = glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,0.0,1.0)));
                var scale = glm.scale(glm.vec3(250,125,250));
                var transformation = (scale)['*'](rotate);
                this.daySky.setModelTransformation(transformation);
                this.daySky.drawit(this.viewM,this.projectionM);
                break;
            
            case 0: //noite
            
                // #***** Sky *****
                var rotate = glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,0.0,1.0)));
                var scale = glm.scale(glm.vec3(250,125,250));
                var transformation = (scale)['*'](rotate);
                this.nightSky.setModelTransformation(transformation);
                this.nightSky.drawit(this.viewM,this.projectionM);
                break;
            
        }
    }
    
    drawscene(view, proj){
               
        //------------------------------------------------------------------------
        
        //rot180 -> ficar virado para a frente
        var rot180 = glm.toMat4(glm.angleAxis(glm.radians(180),glm.vec3(0.0,1.0,0.0))); 
        var rotacion = glm.toMat4(glm.angleAxis(-this.counter/20,glm.vec3(0.0,1.0,0.0)));
        this.carPos = glm.vec3(20*Math.cos(this.counter/20),1,2*20*Math.sin(this.counter/20))
        var transformation = glm.translate(this.carPos);
        
        //transformações primeiro carro
        transformation = (transformation)['*'](rotacion);
        transformation = transformation['*'](rot180);
        this.car.drawit(view, proj, transformation);
        
        //transformações segundo carro
        transformation = glm.translate(glm.vec3(22*Math.cos((this.counter-10)/20), 1, 2*22*Math.sin((this.counter-10)/20)))
                                      ['*'](glm.toMat4(glm.angleAxis(-(this.counter-10)/20,glm.vec3(0.0,1.0,0.0))))
                                      ['*'](glm.toMat4(glm.angleAxis(glm.radians(180),glm.vec3(0.0,1.0,0.0))));
        this.car2.drawit(view, proj, transformation);
        
        
        
        //*****************************        
        // # bancadas 
        transformation = glm.translate(glm.vec3(-90,0,0))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.stand.drawit(view,proj,transformation);
               
        //*******************************************************************************************************************
        // # Billboards and Banners 
        
        //#Billboard  (random textura)
        transformation = glm.translate(glm.vec3(32,0,-50))['*'](glm.toMat4(glm.angleAxis(glm.radians(-10),glm.vec3(0.0,1.0,0.0))));
        this.billboard.drawit(view, proj, transformation);
        
        //#placard (textura dinamica)
        transformation = glm.translate(glm.vec3(-10,-3,-65))['*'](glm.toMat4(glm.angleAxis(glm.radians(-10),glm.vec3(0.0,1.0,0.0))));
        this.placard.drawit(view, proj, transformation);
       
        //# Banners 1 ()
        transformation = glm.translate(glm.vec3(30,0,40))['*'](glm.toMat4(glm.angleAxis(glm.radians(0),glm.vec3(0.0,1.0,0.0))));
        this.finish.drawit(view, proj, transformation);
        
        //# Banners 2 ()
        transformation = glm.translate(glm.vec3(25,0,45))['*'](glm.toMat4(glm.angleAxis(glm.radians(0),glm.vec3(0.0,1.0,0.0))));
        this.finish2.drawit(view, proj, transformation);
        
        //# Banners 3 ()
        transformation = glm.translate(glm.vec3(20,0,50))['*'](glm.toMat4(glm.angleAxis(glm.radians(0),glm.vec3(0.0,1.0,0.0))));
        this.finish3.drawit(view, proj, transformation);
        
      
            
        //******************************************************************************************************************
        // #-----  Floresta de pinheiros +-  ---------------
        
        var rot270 = glm.toMat4(glm.angleAxis(glm.radians(270),glm.vec3(0.0,1.0,0.0)));
        var rot180 = glm.toMat4(glm.angleAxis(glm.radians(180),glm.vec3(0.0,1.0,0.0)));
        var rot90 = glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0)));
        
        transformation = glm.translate(glm.vec3(-100,0.0,80))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot90);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-80,0,70))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot180);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-100,0,-80))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot270);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-80,0,-70))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot90);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(100,0,-80))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot180);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(80,0,-70))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot270);
        this.pineTree.drawit(view, proj, transformation);
       
        transformation = glm.translate(glm.vec3(100,0,80))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot90);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(80,0,70))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot180);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-30,0,70))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot270);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-40,0,90))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot90);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-30,0,-70))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot180);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-40,0,-90))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot270);
        this.pineTree.drawit(view, proj, transformation);
       
        transformation = glm.translate(glm.vec3(30,0,-70))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot90);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(40,0,-90))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot180);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(30,0,70))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot270);
        this.pineTree.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(40,0,90))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        transformation = transformation['*'](rot90);
        this.pineTree.drawit(view, proj, transformation);
        
        //**********************************
        //#** lampadas quadradas ***
        
        transformation = glm.translate(glm.vec3(-20,-3,50))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.Streetlamp.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(20,-3,50))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.Streetlamp.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(20,-3,-50))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.Streetlamp.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-20,-3,-50))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.Streetlamp.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-40,-3,50))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.Streetlamp.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-40,-3,-50))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.Streetlamp.drawit(view, proj, transformation);
        
        
        //**********************************
        //#** lampadas esfericas ***
        
        transformation = glm.translate(glm.vec3(-40,-3,70))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.Groundlamp.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(35,-3,100))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.Groundlamp.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(85,-3,-96))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.Groundlamp.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-56,-3,-43))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.Groundlamp.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-45,-3,94))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.Groundlamp.drawit(view, proj, transformation);
        
        transformation = glm.translate(glm.vec3(-49,-3,-56))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.Groundlamp.drawit(view, proj, transformation);
        
        
        //*******************************     
        // #------Ground ------
        var scale = glm.scale(glm.vec3(250,0.05,250));
        transformation = glm.translate(glm.vec3(0,-0.2,0));
        transformation = transformation['*'](scale);
        this.chao.setModelTransformation(transformation);
        this.chao.drawit(view, proj);
        
        
        // #----- Sky --------
        var rotate = glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,0.0,1.0)));
        var scale = glm.scale(glm.vec3(250,125,250));
        transformation = (scale)['*'](rotate);
        this.nightSky.setModelTransformation(transformation);
        this.nightSky.drawit(view, proj); 
                         
    }
  
    //==========================================================================================================================
    
    keyprocess(evt){
        console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
         switch (evt.keyCode) {
            case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
            
            case 97: // #a 
                this.theta+=Math.PI/20;
                break;
                 
            case 98: //b
                var red = 0.0; 
                var blue = 1.0;
                var green = 0.0;
                break;
             
            case 100: //d daySky
                this.changeCenario(1);
                
                break;
             
             
            case 103: //g
                var red = 0.0; 
                var blue = 0.0;
                var green = 1.0;
                break;
             
            case 108: // #l
                this.posx -= Math.sin(this.theta);
                this.posz -= Math.cos(this.theta);
                break;
             
                        
             case 112: // #p
                this.posx += Math.sin(this.theta);
                this.posz += Math.cos(this.theta);
                break;
            
             case 114: //r
                var red = 1.0; 
                var blue = 0.0;
                var green = 0.0;
                break;

             case 115: // #s
                this.theta-=Math.PI/20;
                break;
                      
             case 118: // #v
                 this.viewpoint = !this.viewpoint;
                 break;
             
             case 119: //w
                var red = 1.0; 
                var blue = 1.0;
                var green = 1.0;
                             
            case 110: //n nightSky
                this.changeCenario(0);
                
                break;
        }
    }
      
    //=========================================================================================================================
    //=========================================================================================================================
    // # --------------------- B.4 Optional ----------------------------
    
    render(){
        this.counter++;
        if (this.viewpoint){ //#  espectador
            /*this.viewM = glm.lookAt(glm.vec3(this.posx,5,this.posz),
                                glm.vec3(Math.sin(this.theta)+this.posx, 5.0, Math.cos(this.theta)+this.posz),
                                glm.vec3(0,1,0));*/
            //*******************************************************************************************************************
            // #Camera calculations for stereoscopic visualization
            //*******************************************************************************************************************
            //Camera calculations
            this.pcam=glm.vec3(this.posx,5.0,this.posz);//posicao inicial da camera
            this.pat= glm.vec3(Math.sin(this.theta)+this.posx,5.0,Math.cos(this.theta)+this.posz);//posição da camera está a apontar
            this.vup=glm.vec3(0,1,0);

            this.vca=this.pat['-'](this.pcam);
            //console.log("vca:"+this.vca);

            this.vright=glm.cross(this.vca,this.vup);
            //console.log("vright:"+this.vright);

            var deye=0.01;
            this.aux= (glm.normalize(this.vright))['*'](deye/2);
            //console.log("aux:"+this.aux);
            this.pleft= this.pcam['-'](this.aux);
            this.prigth= this.pcam['+'](this.aux);


            console.log("pleft:" +this.pleft);
            //console.log("pright:"+ this.prigth);

            this.viewMleft= glm.lookAt(this.pleft,this.pat,glm.vec3(0,1,0));
            this.viewMright= glm.lookAt(this.prigth,this.pat,glm.vec3(0,1,0));
        }else{   //#  condutor
            /*this.viewM = glm.lookAt(glm.vec3(this.carPos[0],this.carPos[1]+6,this.carPos[2]),
                                glm.vec3(this.carPos[0]+Math.cos(-this.counter/20-Math.PI/2), 6.5,
                                             this.carPos[2]-Math.sin(-this.counter/20-Math.PI/2)),
                                glm.vec3(0,2,0));*/
                                
            //*******************************************************************************************************************
            // #Camera calculations for stereoscopic visualization-for driver
            //*******************************************************************************************************************
              
            this.pcam=glm.vec3(this.carPos[0],this.carPos[1]+6,this.carPos[2]);//posicao inicial da camera
            this.pat= glm.vec3(this.carPos[0]+Math.cos(-this.counter/20-Math.PI/2), 6.5,this.carPos[2]-Math.sin(-this.counter/20-Math.PI/2));//posição da camera está a apontar
            this.vup= glm.vec3(0,2,0);
            
            var deye=0.01;  
              
            this.vca=this.pat['-'](this.pcam);
            this.vright=glm.cross(this.vca,this.vup);
            this.aux= (glm.normalize(this.vright))['*'](deye/2);
            
            this.pleft= this.pcam['-'](this.aux);
            console.log("camera condutor:"+this.pleft);
            this.prigth= this.pcam['+'](this.aux);
              
            this.viewMleft= glm.lookAt(this.pleft,this.pat,glm.vec3(0,1,0));
            this.viewMright= glm.lookAt(this.prigth,this.pat,glm.vec3(0,1,0));
              
        }


        /*this.viewMirror = glm.lookAt(glm.vec3(this.carPos[0],this.carPos[1]+6,this.carPos[2]),
                                glm.vec3(this.carPos[0]+Math.cos(-this.counter/20-Math.PI/2), 6.5,
                                             this.carPos[2]-Math.sin(-this.counter/20-Math.PI/2)),
                                glm.vec3(0,2,0));*/
                          
        //*******************************************************************************************************************
        // #Camera calculations for stereoscopic visualization-for placard video
        //*******************************************************************************************************************
        
        this.pcam=glm.vec3(this.carPos[0],this.carPos[1]+6,this.carPos[2]);//posicao inicial da camera
        this.pat= glm.vec3(this.carPos[0]+Math.cos(-this.counter/20-Math.PI/2), 6.5,this.carPos[2]-Math.sin(-this.counter/20-Math.PI/2));//posição da camera está a apontar
        this.vup= glm.vec3(0,2,0);
            
        var deye=0.01;  
              
        this.vca=this.pat['-'](this.pcam);
        this.vright=glm.cross(this.vca,this.vup);
        this.aux= (glm.normalize(this.vright))['*'](deye/2);
            
        this.pleft= this.pcam['-'](this.aux);
        console.log("camera condutor:"+this.pleft);
        this.prigth= this.pcam['+'](this.aux);
              
        this.viewMleftp= glm.lookAt(this.pleft,this.pat,glm.vec3(0,1,0));
        this.viewMrightp= glm.lookAt(this.prigth,this.pat,glm.vec3(0,1,0));

        // this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.gl.enable(this.gl.DEPTH_TEST);
        // render to our targetTexture by binding the framebuffer
        this.gl.bindFramebuffer(this.gl.FRAMEBUFFER, this.fb);
        this.gl.framebufferTexture2D(this.gl.FRAMEBUFFER, this.gl.COLOR_ATTACHMENT0, 
                                   this.gl.TEXTURE_2D, this.targetTexture, 0);
        
        // Tell WebGL how to convert from clip space to pixels
        this.gl.viewport(0, 0, this.targetTextureWidth, this.targetTextureHeight);


        this.gl.clearColor(0, 0, 1, 1);   // clear to blue
        this.gl.colorMask(1.0,1.0,1.0,1.0);
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);

        // normally the view matrix should be different   
        this.drawscene(this.viewMleftp,this.projectionM);
        
        this.gl.clear(this.gl.DEPTH_BUFFER_BIT);
        //#CAMERA -> LADO DIREITO
        this.gl.colorMask(0.0,1.0,1.0,1.0);
        this.drawscene(this.viewMrightp,this.projectionM);
   
        // now going to render to the canvas
        
        this.gl.bindFramebuffer(this.gl.FRAMEBUFFER, null);
        this.gl.enable(this.gl.DEPTH_TEST);

        this.gl.viewport(0, 0, this.gl.canvas.width, this.gl.canvas.height);
        
        // set view for the user
        // Tell WebGL how to convert from clip space to pixels
        //*****************************************************************************************************************
        //*********************************************************************************************************************
        this.gl.clearColor(0.0, 0.0, 0.0, 1);   // clear to blue
        this.gl.colorMask(1.0,1.0,1.0,1.0);
        this.gl.clear(this.gl.DEPTH_BUFFER_BIT);
        
        //#CAMERA -> LADO ESQUERDO
        this.gl.colorMask(1.0,0.0,0.0,1.0);
        this.drawscene(this.viewMleft,this.projectionM);
        
        this.gl.clear(this.gl.DEPTH_BUFFER_BIT);
        //#CAMERA -> LADO DIREITO
        this.gl.colorMask(0.0,1.0,1.0,1.0);
        this.drawscene(this.viewMright,this.projectionM);
        

    }
}

var app = new my5app('my5Canvas');

app.run();

</script>